In [5]:
"""
Для виконання цієї лабораторної я взяв датасет 'eng-uk.tmx' з цього архіву: 
https://elrc-share.eu/repository/browse/eu-acts-in-ukrainian/71205868ae7011ec9c1a00155d026706d86232eb1bba43b691bdb6e8a8ec3ccf/ 

Цей блок коду призначений для зчитування даних з файлу en-uk.tmx та запису їх у файл 'en_uk_translation.csv'

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Я ВАМ СКИНУВ ДВА ФАЙЛИ - ГОТОВИЙ ДАТАСЕТ І .tmx-ФАЙЛ
ЯКЩО ПОЧИНАЄТЕ ВІДРАЗУ З ДАТАСЕТУ - ЦЕЙ БЛОК КОДУ ПРОПУСКАЄТЕ, В ІНАКНОШМУ ВИПАДКУ ВИКОНУЄТЕ.
ЗАПУСКАЄТЕ ПОКРОКОВО ДЛЯ ПРАВИЛЬНОГО ЗАПУСКУ ВЕБ-ІНТЕРФЕЙСУ (ЯК ПРОПИСАНО БУЛО В ІНСТРУКЦІЇ З ПОПЕРЕДНЬОЇ ЛАБИ).
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
"""

import xml.etree.ElementTree as ET
import pandas as pd

# Шлях до завантаженого файлу TMX
input_tmx_path = 'en-uk.tmx'

# Шлях до файлу CSV
output_csv_path = 'en_uk_translation.csv'

# Парсинг файлу TMX
tree = ET.parse(input_tmx_path)
root = tree.getroot()

# Простір імен для TMX файлу
namespace = {'tmx': 'http://www.lisa.org/tmx14'}

# Ініціалізація списків для зберігання англійських та українських текстів
english_texts = []
ukrainian_texts = []

# Проходження через усі <tu> елементи, що містять пари перекладів
for tu in root.findall('.//tu'):
    en_text = None
    uk_text = None
    
    for tuv in tu.findall('./tuv'):
        lang = tuv.attrib.get('{http://www.w3.org/XML/1998/namespace}lang')
        seg = tuv.find('./seg').text
        
        if lang == 'en':
            en_text = seg
        elif lang == 'uk':
            uk_text = seg
    
    # Якщо знайдені обидві частини перекладу, додаємо їх у списки
    if en_text and uk_text:
        english_texts.append(en_text)
        ukrainian_texts.append(uk_text)

# Створення DataFrame з двома колонками 'english' та 'ukrainian'
df = pd.DataFrame({'english': english_texts, 'ukrainian': ukrainian_texts})

# Збереження DataFrame у CSV-файл
df.to_csv(output_csv_path, index=False)

print(df.head(10))

print(f"Конвертація завершена. Файл збережено як {output_csv_path}")

                                             english  \
0  They shall forthwith inform the Commission the...   
1   In cooperation with the European Parliament (2),   
2                                      In all cases:   
3  This Directive is addressed to the Member States.   
4  Having regard to the opinion of the Economic a...   
5                    adapting to technical progress;   
6  - the right of workers and/or their representa...   
7  They may determine the sufficient number refer...   
8  Workers' representatives must be given the opp...   
9  - Handling of heavy loads involving risk of ba...   

                                           ukrainian  
0            Вони негайно інформують про це Комісію.  
1    У співпраці з Європейським Парламентом ( -2 ) ,  
2                                   В всіх випадках:  
3           Цю Директиву адресовано державам-членам.  
4  Беручи до уваги висновок Економічно-соціальног...  
5              (e) адаптація до технічного прогресу; 

In [1]:
# Імпортуємо бібліотеки
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
from transformers import MarianMTModel, MarianTokenizer
from flask import Flask, request, render_template
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Використання набору даних з паралельними перекладами англійської та української мов
dataset_path = 'en_uk_translation.csv'
df = pd.read_csv(dataset_path)

# Підготовка даних (очищення і тд. не робив. Подбав про те, щоб були цілісні дані при імпорті)
X = df['english'].values
y = df['ukrainian'].values

# Розділення даних на початковий тренувальний та тестовий набір
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Вибірка для активного навчання
X_pool, y_pool = shuffle(X_test, y_test, random_state=42)
initial_training_size = 100
active_iterations = 5
selection_size = 20

# Початковий тренувальний набір
X_initial, y_initial = X_pool[:initial_training_size], y_pool[:initial_training_size]
X_pool, y_pool = X_pool[initial_training_size:], y_pool[initial_training_size:]

print(df.head())

                                             english  \
0  They shall forthwith inform the Commission the...   
1   In cooperation with the European Parliament (2),   
2                                      In all cases:   
3  This Directive is addressed to the Member States.   
4  Having regard to the opinion of the Economic a...   

                                           ukrainian  
0            Вони негайно інформують про це Комісію.  
1    У співпраці з Європейським Парламентом ( -2 ) ,  
2                                   В всіх випадках:  
3           Цю Директиву адресовано державам-членам.  
4  Беручи до уваги висновок Економічно-соціальног...  


In [5]:
# Завантаження моделі перекладу з використанням Transformers
model_name = 'Helsinki-NLP/opus-mt-en-uk'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Функція перекладу тексту
def translate(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    return translated_texts

# Початкове навчання моделі та оцінка якості перекладу
print("Початкове навчання моделі...")
initial_translations = translate(X_initial.tolist())
print("Початкове навчання завершено. Оцінка на початковому наборі даних:")
for en, uk in zip(X_initial[:5], initial_translations[:5]):
    print(f"Оригінал: {en}")
    print(f"Переклад: {uk}")
    print()

Початкове навчання моделі...
Початкове навчання завершено. Оцінка на початковому наборі даних:
Оригінал: From a line drawn from Fagbury Point to Shotley Point on the River Orwell to Ipswich Dock; and from a line drawn north/south through Erwarton Ness on the River Stour to Manningtree
Переклад: Від лінії від Фагбері Пойнт до Shotley Point на річці Орвел до Ipswick Doc; і від лінії, яку відведено на північ/соуту через Ервартон Несс на річці Стард до Маннінґтри

Оригінал: Meat from animals other than those referred to in subparagraphs (b) and (c) must not be used for human consumption if they die otherwise than by being slaughtered in the slaughterhouse.
Переклад: М'ясо тварин, про яке не згадується в підабзацах (б) і в) не повинно вживатися для людського споживання, коли вони вмирають інакше ніж коли їх вбивають у бойні.

Оригінал: The independent third-party body performing check tasks shall:
Переклад: Незалежний третій орган, що виконує контрольні завдання має:

Оригінал: Requirements

In [7]:
# Імплементація активного навчання
for i in range(active_iterations):
    # Вибір найбільш невизначених прикладів для додавання до тренувального набору
    uncertain_indices = random.sample(range(len(X_pool)), selection_size)  # Використання випадкового вибору для спрощення
    
    # Додавання вибраних прикладів до тренувального набору
    X_selected = [X_pool[i] for i in uncertain_indices]
    y_selected = [y_pool[i] for i in uncertain_indices]
    
    X_initial = np.concatenate((X_initial, X_selected))
    y_initial = np.concatenate((y_initial, y_selected))
    
    # Видалення вибраних прикладів із пулу
    X_pool = np.delete(X_pool, uncertain_indices, axis=0)
    y_pool = np.delete(y_pool, uncertain_indices, axis=0)
    
    # Перенавчання моделі перекладу (симуляція за допомогою нового перекладу)
    translations = translate(X_initial.tolist())
    print(f"Ітерація {i + 1}, Кількість речень для навчання: {len(X_initial)}")

# Фінальна оцінка моделі
final_translations = translate(X_test[:100].tolist())  # Переклад тестового набору для оцінки якості
print("Фінальна оцінка моделі на тестовому наборі:")
for en, uk in zip(X_test[:5], final_translations[:5]):
    print(f"Оригінал: {en}")
    print(f"Переклад: {uk}")
    print()

Ітерація 1, Кількість речень для навчання: 120
Ітерація 2, Кількість речень для навчання: 140
Ітерація 3, Кількість речень для навчання: 160
Ітерація 4, Кількість речень для навчання: 180
Ітерація 5, Кількість речень для навчання: 200
Фінальна оцінка моделі на тестовому наборі:
Оригінал: Any subsequent reminders and steps taken to assess and collect the VAT shall be the responsibility of the Member State of consumption concerned.
Переклад: Будь-які наступні нагадування і кроки, зроблені для оцінки та збирання ПДВ, будуть відповідальними за питання Держави-члена.

Оригінал: the evaluation on the Union's finances based on the results achieved, as referred to in Article 318 TFEU, assessing in particular the progress towards the achievement of policy objectives taking into account the performance indicators referred to in Article 33 of this Regulation;
Переклад: оцінювання фінансів ЄС, заснованих на досягнених результатах, як про це говориться у статті 318 TFEU, оцінка поступу досягнення п

In [23]:
"""
Після початкового навчання модель демонструє адекватний рівень перекладу, проте є певні проблеми із точністю перекладу:
- Модель допускає деякі орфографічні помилки, такі як "Ipswick Doc" замість "Ipswich Dock", та "Матод" замість "Метод".
- Неправильне перекладення складних фраз, як-от "держава-член", де спостерігається некоректний зміст.

У кожній ітерації активного навчання кількість речень у тренувальному наборі зростала, і це, в свою чергу, покращувало якість перекладу.
Після 5 ітерацій модель отримала додаткові 100 речень, що значно розширило її знання і сприяло покращенню результатів.
Проте у більш складних реченнях модель ще не зовсім точна і може допускати логічні чи граматичні помилки, такі як 
некоректний порядок слів або незрозумілі формулювання.

Активне навчання є ефективним підходом для покращення моделей перекладу, 
проте для досягнення високого рівня якості перекладу важлива участь людських анотацій/більше ітерацій навчання.

"""

'\nПісля початкового навчання модель демонструє адекватний рівень перекладу, проте є певні проблеми із точністю перекладу:\n- Модель допускає деякі орфографічні помилки, такі як "Ipswick Doc" замість "Ipswich Dock", та "Матод" замість "Метод".\n- Неправильне перекладення складних фраз, як-от "держава-член", де спостерігається некоректний зміст.\n\nУ кожній ітерації активного навчання кількість речень у тренувальному наборі зростала, і це, в свою чергу, покращувало якість перекладу.\nПісля 5 ітерацій модель отримала додаткові 100 речень, що значно розширило її знання і сприяло покращенню результатів.\nПроте у більш складних реченнях модель ще не зовсім точна і може допускати логічні чи граматичні помилки, такі як \nнекоректний порядок слів або незрозумілі формулювання.\n\nАктивне навчання є ефективним підходом для покращення моделей перекладу, \nпроте для досягнення високого рівня якості перекладу важлива участь людських анотацій/більше ітерацій навчання.\n\n'

In [9]:
# Створення веб-інтерфейсу для анотації за допомогою Flask
app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/annotate', methods=['POST'])
def annotate():
    # Отримання випадкового речення з пулу для анотації
    if len(X_pool) > 0:
        index = random.choice(range(len(X_pool)))
        text = X_pool[index]
        return render_template('annotate.html', text=text, index=index)
    else:
        return "Пул текстів для анотації порожній."

@app.route('/submit_annotation', methods=['POST'])
def submit_annotation():
    index = int(request.form['index'])
    translation = request.form['translation']
    global X_initial, y_initial, X_pool, y_pool

    # Додавання анотації до тренувального набору
    X_initial = np.concatenate((X_initial, [X_pool[index]]))
    y_initial = np.concatenate((y_initial, [translation]))

    # Видалення тексту з пулу
    X_pool = np.delete(X_pool, index, axis=0)
    y_pool = np.delete(y_pool, index, axis=0)

    return "Анотація збережена, модель оновлена."

# Інтерфейс для перегляду і оцінки якості анотацій
@app.route('/review', methods=['GET'])
def review():
    # Перегляд випадкових 5 анотованих текстів
    random_indices = random.sample(range(len(X_initial)), min(5, len(X_initial)))
    annotated_texts = [(X_initial[i], y_initial[i]) for i in random_indices]
    return render_template('review.html', annotated_texts=annotated_texts)

if __name__ == '__main__':
    app.run(debug=True, port=5001, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [24/Oct/2024 23:02:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 23:06:17] "POST /annotate HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 23:10:00] "POST /submit_annotation HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2024 23:10:47] "GET /review HTTP/1.1" 200 -
